# Projet SDP : Explications de classements par Trade-offs

**Membres du groupe** : Adham Noureldin, Aymane Chalh, Daphné Maschas

**Date** : 19 Janvier 2026

In [ ]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

## 1. Introduction

Ce projet vise à fournir des explications intelligibles pour le classement des candidats au concours de l'internat de médecine. Au lieu d'utiliser la somme pondérée brute, nous justifions qu'un candidat $x$ est meilleur qu'un candidat $y$ en montrant comment ses points forts compensent ses points faibles via des modèles d'optimisation (MIP).

## 2. Préparation des données

Dans cette section, nous définissons les poids des matières et les notes des candidats de référence.

In [3]:
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic Pathology", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes des candidats 
notes_x = [85, 81, 71, 69, 75, 81, 88] # Xavier
notes_y = [81, 81, 75, 63, 67, 88, 95] # Yvonne
notes_z = [74, 89, 74, 81, 68, 84, 79]
notes_t = [74, 71, 84, 91, 77, 76, 73]
notes_w = [79, 69, 78, 76, 67, 84, 79]
notes_w_prime = [57, 76, 81, 76, 82, 86, 77]
notes_u = [72, 66, 75, 85, 88, 66, 93]
notes_v = [71, 73, 63, 92, 76, 79, 93]

# Autres candidats
notes_a1 = [89, 74, 81, 68, 84, 79, 77]
notes_a2 = [71, 84, 91, 79, 78, 73.5, 77] # 73.5 pour Forensic

Fonctions utilitaires:

In [4]:
def preparer_donnees(notes_a, notes_b, poids):
    """Calcule les contributions et sépare les Pros et les Cons."""
    contribs = [poids[i] * (notes_a[i] - notes_b[i]) for i in range(len(poids))]
    P = [i for i, c in enumerate(contribs) if c > 0]
    C = [i for i, c in enumerate(contribs) if c < 0]
    return contribs, P, C

def afficher_resultats(titre, status, explication):
    """Affiche proprement le résultat d'un modèle d'optimisation."""
    print(f"\n=== {titre} ===")
    if status == "OPTIMAL" and explication:
        print("SUCCÈS : Une explication a été trouvée :")
        for groupe in explication:
            print(f"  • {groupe}")
    else:
        print("ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.")
    print("-" * 50)

## 3. Question 1 : Explication de type (1-1)

**Démarche** : Nous cherchons à coupler chaque critère défavorable à un unique critère favorable qui le compense individuellement.

**Contrainte** : Chaque "Pro" est utilisé au plus une fois, chaque "Con" est couvert une fois.

**Condition** : $c_{pro} + c_{con} > 0$.

In [5]:
def explication_1_1(notes_a, notes_b, poids, features):
    """
    Calcule une explication de type (1-1) où chaque point fort compense un unique point faible.
    
    Args:
        notes_a (list): Notes du candidat supérieur.
        notes_b (list): Notes du candidat inférieur.
        
    Returns:
        tuple: (status, explication) où status est "OPTIMAL" ou "INFEASIBLE" 
               et explication est une liste de chaînes décrivant les binômes.
    """
    contribs, P, C = preparer_donnees(notes_a, notes_b, poids)
    
    m = gp.Model("Q1_1_1")
    m.Params.OutputFlag = 0
    
    # x[i,j] = 1 si le pro i compense le con j
    x = m.addVars(P, C, vtype=GRB.BINARY, name="x")
    
    # Contraintes : Couplage parfait
    for j in C:
        m.addConstr(gp.quicksum(x[i,j] for i in P) == 1)
    for i in P:
        m.addConstr(gp.quicksum(x[i,j] for j in C) <= 1)
        
    # Validité : pro + con > 0
    for i in P:
        for j in C:
            if contribs[i] + contribs[j] <= 0:
                m.addConstr(x[i,j] == 0)

    m.optimize()
    
    if m.Status == GRB.OPTIMAL:
        res = [f"{features[i]} (+{contribs[i]}) compense {features[j]} ({contribs[j]})" 
                       for i in P for j in C if x[i,j].X > 0.5]
        return "OPTIMAL", res
    return "INFEASIBLE", None

In [6]:
status, explication = explication_1_1(notes_x, notes_y, poids, matieres)
afficher_resultats("QUESTION 1 (1-1) : Xavier vs Yvonne", status, explication)

Restricted license - for non-production use only - expires 2027-11-29

=== QUESTION 1 (1-1) : Xavier vs Yvonne ===
SUCCÈS : Une explication a été trouvée :
  • Anatomie (+32) compense Chirurgie (-28)
  • Diagnostic (+36) compense Forensic Pathology (-35)
  • Epidemiologie (+48) compense Génétique (-42)
--------------------------------------------------


## 4. Question 2: Explication (1-m)

**Démarche** : Nous généralisons le modèle.

**Type (1-m)** : Un avantage puissant peut compenser plusieurs faiblesses. C'est utile quand un candidat excelle dans une matière à gros coefficient.

In [7]:
def explication_1_m(notes_a, notes_b, poids, features, epsilon=1e-4):
    """
    Calcule une explication de type (1-m) où un point fort peut compenser plusieurs points faibles.
    
    Args:
        notes_a (list): Notes du candidat supérieur.
        notes_b (list): Notes du candidat inférieur.
        poids (list): Coefficients des matières.
        features (list): Noms des matières.
        epsilon (float): Seuil de supériorité pour le trade-off.
        
    Returns:
        tuple: (status, explication) où explication est une liste de chaînes.
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b, poids)
    
    # Modèle Gurobi
    m = gp.Model("Explication_1_m")
    m.setParam('OutputFlag', 0) # Mode silencieux

    # Variables
    # x[i,j] = 1 si Pro i couvre Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"link_{i}_{j}")
    
    # y[i] = 1 si Pro i est utilisé
    y = {}
    for i in P:
        y[i] = m.addVar(vtype=GRB.BINARY, name=f"use_{i}")

    # Contrainte C1: Chaque Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x[i, j] for i in P) == 1, name=f"cover_{j}")

    # Contrainte C2: Lien x_ij <= y_i
    for i in P:
        for j in C:
            m.addConstr(x[i, j] <= y[i], name=f"link_logic_{i}_{j}")

    # Contrainte C3: Somme pondérée positive pour chaque groupe (1-m)
    # delta_i * y_i + sum(contribs_j * x_ij) >= epsilon * y_i
    for i in P:
        sum_cons = gp.quicksum(contribs[j] * x[i, j] for j in C)
        m.addConstr(contribs[i] * y[i] + sum_cons >= epsilon * y[i], name=f"validity_{i}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        for i in P:
            if y[i].X > 0.5:
                cons = [features[j] for j in C if x[i, j].X > 0.5]
                res.append(f"{features[i]} compense le groupe {cons}")
        return "OPTIMAL", res
    elif m.Status == GRB.INFEASIBLE:
        return "INFEASIBLE", None
    else:
        return f"Status {m.Status}", None

In [8]:
status, result = explication_1_m(notes_w, notes_w_prime, poids, matieres)
afficher_resultats("QUESTION 2 (1-m) : w vs w'", status, result)


=== QUESTION 2 (1-m) : w vs w' ===
SUCCÈS : Une explication a été trouvée :
  • Anatomie compense le groupe ['Biologie', 'Chirurgie', 'Epidemiologie', 'Forensic Pathology']
  • Génétique compense le groupe []
--------------------------------------------------


In [9]:
status, result = explication_1_m(notes_u, notes_v, poids, matieres)
afficher_resultats("QUESTION 2 (1-m) : u vs v", status, result)


=== QUESTION 2 (1-m) : u vs v ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 5. Question 3: Explication (m-1)

**Démarche** : Nous généralisons le modèle.

**Type (m-1)** : Plusieurs petits avantages s'unissent pour compenser une faiblesse majeure.

In [10]:
def explication_m_1(candidate_sup, candidate_inf, notes_a, notes_b, poids, features):
    """
    Calcule une explication de type (m-1) où plusieurs points forts s'unissent 
    pour compenser un seul point faible.
    
    Args:
        notes_a (list): Notes du candidat supérieur.
        notes_b (list): Notes du candidat inférieur.
        poids (list): Coefficients des matières.
        features (list): Noms des matières.
        
    Returns:
        tuple: (status, explication) où explication est une liste de chaînes.
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b, poids)

    # Modèle Gurobi
    m = gp.Model(f"Explication_m_1_{candidate_sup}_{candidate_inf}")
    m.setParam('OutputFlag', 0)

    # Variables x[i, j] : Pro i aide à couvrir Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"x_{features[i]}_{features[j]}")

    # Contrainte C1: Chaque Pro utilisé au max 1 fois
    for i in P:
        m.addConstr(gp.quicksum(x[i, j] for j in C) <= 1, name=f"disjoint_{features[i]}")

    # Contrainte C2: Chaque Con doit être compensé (Somme Pros + Con >= 0)
    for j in C:
        m.addConstr(gp.quicksum(contribs[i] * x[i, j] for i in P) + contribs[j] >= 0, 
                    name=f"cover_{features[j]}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        for j in C:
            avantages = [features[i] for i in P if x[i,j].X > 0.5]
            res.append(f"L'union de {avantages} compense {features[j]} ({contribs[j]})")
        return "OPTIMAL", res
    return "INFEASIBLE", None

In [11]:
status, result = explication_m_1("y", "z", notes_y, notes_z, poids, matieres)
afficher_resultats("QUESTION 3 (m-1) : Candidat y vs z", status, result)


=== QUESTION 3 (m-1) : Candidat y vs z ===
SUCCÈS : Une explication a été trouvée :
  • L'union de ['Anatomie'] compense Biologie (-56)
  • L'union de ['Forensic Pathology', 'Génétique'] compense Diagnostic (-108)
  • L'union de ['Chirurgie'] compense Epidemiologie (-6)
--------------------------------------------------


In [12]:
status, result = explication_m_1("z", "t", notes_z, notes_t, poids, matieres)
afficher_resultats("QUESTION 3 (m-1) : Candidat z vs t", status, result)


=== QUESTION 3 (m-1) : Candidat z vs t ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 6. Question 4: Modèle Hybride

**Démarche** : Certains cas, comme $u > v$, ne possèdent pas d'explication simple. Nous implémentons un modèle hybride qui autorise une partition arbitraire des critères en groupes (1-m) ou (m-1). Nous utilisons une formulation avec des variables binaires et des contraintes de type "Big-M" pour activer les trade-offs.

In [13]:
def explication_hybride(notes_a, notes_b, poids, features):
    """
    Résout le problème d'explication mixte (Question 4).
    Autorise la création d'une partition flexible combinant des structures (1-m) et (m-1).
    
    Args:
        notes_a (list): Notes du candidat supérieur.
        notes_b (list): Notes du candidat inférieur.
        epsilon (float): Seuil de supériorité pour valider un groupe.
        
    Returns:
        tuple: (status, explication) où explication est une liste de chaînes de caractères.
    """
    # Calcul des contributions
    contribs, P, C = preparer_donnees(notes_a, notes_b, poids)

    # Modèle
    m = gp.Model("Mixed_Explanation")
    m.setParam('OutputFlag', 0)
    
    # Variables de liaison pour les deux types de structures
    x_1m = {} # x_1m[i,j] = 1 si le Pro i (Hub) couvre le Con j
    x_m1 = {} # x_m1[i,j] = 1 si le Con j (Hub) est couvert par le Pro i

    for i in P:
        for j in C:
            x_1m[i, j] = m.addVar(vtype=GRB.BINARY, name=f"1m_{features[i]}_{features[j]}")
            x_m1[i, j] = m.addVar(vtype=GRB.BINARY, name=f"m1_{features[i]}_{features[j]}")

    # Indicateurs de rôle "Hub"
    # h_P[i] = 1 si le Pro i est utilisé comme centre d'un (1-m)
    h_P = {i: m.addVar(vtype=GRB.BINARY) for i in P}
    
    # h_C[j] = 1 si le Con j est utilisé comme centre d'un (m-1)
    h_C = {j: m.addVar(vtype=GRB.BINARY) for j in C}

    # Contraintes
    # Contrainte C1: Tout Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x_1m[i, j] for i in P) + h_C[j] == 1, name=f"Cover_{j}")

    # Contrainte C2: Tout Pro utilisé au max une fois
    for i in P:
        m.addConstr(h_P[i] + gp.quicksum(x_m1[i, j] for j in C) <= 1, name=f"Use_{i}")

    # Contrainte C3: Cohérence logique des liens
    for i in P:
        for j in C:
            m.addConstr(x_1m[i, j] <= h_P[i], name=f"Logic_1m_{i}_{j}")
            m.addConstr(x_m1[i, j] <= h_C[j], name=f"Logic_m1_{i}_{j}")

    # Contrainte C4: Validité des Trade-offs (Somme pondérée > 0)
    
    for i in P: # Pour chaque Pro Hub (Structure 1-m): delta_i + sum(delta_j * x_1m_ij) >= 0
        contrib_cons = gp.quicksum(contribs[j] * x_1m[i, j] for j in C)
        m.addConstr(contribs[i] * h_P[i] + contrib_cons >= 0, name=f"Valid_1m_{i}")
    
    for j in C: # Pour chaque Con Hub (Structure m-1): sum(delta_i * x_m1_ij) + delta_j >= 0
        contrib_pros = gp.quicksum(contribs[i] * x_m1[i, j] for i in P)
        m.addConstr(contrib_pros + contribs[j] * h_C[j] >= 0, name=f"Valid_m1_{j}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        # Structures (1-m)
        for i in P:
            if h_P[i].X > 0.5:
                cons_covered = [features[j] for j in C if x_1m[i, j].X > 0.5]
                if cons_covered:
                    res.append(f"Groupe (1-m) : L'avantage {features[i]} (+{contribs[i]}) compense {cons_covered}")
        
        # Structures (m-1)
        for j in C:
            if h_C[j].X > 0.5:
                pros_used = [features[i] for i in P if x_m1[i, j].X > 0.5]
                val_pros = sum(contribs[i] for i in P if x_m1[i, j].X > 0.5)
                res.append(f"Groupe (m-1) : Les avantages {pros_used} (+{val_pros}) compensent {features[j]} ({contribs[j]})")
        
        return "OPTIMAL", res
    
    return "INFEASIBLE", None

In [14]:
status, result = explication_hybride(notes_z, notes_t, poids, matieres)
afficher_resultats("QUESTION 4 (hybride) : Candidat z vs t", status, result)


=== QUESTION 4 (hybride) : Candidat z vs t ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage Biologie (+126) compense ['Diagnostic', 'Epidemiologie']
  • Groupe (m-1) : Les avantages ['Forensic Pathology', 'Génétique'] (+76) compensent Chirurgie (-70)
--------------------------------------------------


In [15]:
status, result = explication_hybride(notes_a1, notes_a2, poids, matieres)
afficher_resultats("QUESTION 4 (hybride) : Candidat a1 vs a2", status, result)


=== QUESTION 4 (hybride) : Candidat a1 vs a2 ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------


## 7. Application aux données réelles

### 1. Dataset Breast Cancer

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import random

L'idée est de passer d'une approche boîte noire (la Régression Logistique qui prédit un diagnostic) à une approche explicable (justifier pourquoi le patient A est considéré comme plus sain que le patient B).

Le modèle de décision ici n'est pas donné a priori. On commence donc par entraîner une Régression Logistique sur le dataset. Afin d'en extraire les coefficients ($w$). 

Ce sont ces poids qui représentent l'importance accordée par l'IA à chaque critère (taille de la cellule, épaisseur, etc.)

In [38]:
breastcancer_data = pd.read_csv("data/breastcancer_processed.csv")
breastcancer_data.head()
breastcancer_X, breastcancer_y = breastcancer_data.drop(columns="Benign"), breastcancer_data["Benign"]
breastcancer_feature_names = breastcancer_X.columns.tolist()

Il faut ensuite standardiser les données (on va utiliser StandardScaler), pour éviter qu'un critère avec de grandes valeurs numériques écrase les autres, et ce qui ferait que les explications de trade-off ne voudront rien dire.

In [20]:
scaler = StandardScaler()
breastcancer_X_scaled = scaler.fit_transform(breastcancer_X)

model = LogisticRegression(max_iter=10000)
model.fit(breastcancer_X_scaled, breastcancer_y)

breastcancer_weights = model.coef_[0] 

In [47]:
def evaluer_explicabilite_dataset(X, poids, names, n_paires=100):
    """
    Echantillonne des paires (x, y) et calcule la proportion d'échecs d'explication.
    """
    scores = X @ poids
    indices = list(range(len(X)))
    paires_testees = 0
    echecs = 0

    print(f"Analyse de {n_paires} paires aléatoires...")
    
    while paires_testees < n_paires:
        # On tire deux patients au hasard
        idx_a, idx_b = random.sample(indices, 2)
        
        # On s'assure de comparer le meilleur au moins bon
        if scores[idx_a] >= scores[idx_b]:
            patient_sup, patient_inf = X[idx_a], X[idx_b]
        else:
            patient_sup, patient_inf = X[idx_b], X[idx_a]
            
        # Tentative d'explication avec la fonction hybride
        status, _ = explication_hybride(patient_sup, patient_inf, poids, names)
        
        if status != "OPTIMAL":
            echecs += 1
        
        paires_testees += 1

    taux_echec = (echecs / n_paires) * 100
    return echecs, n_paires, taux_echec

n_samples = 100
nb_echecs, total, proportion = evaluer_explicabilite_dataset(breastcancer_X_scaled, breastcancer_weights, breastcancer_feature_names, n_samples)

print(f"\n=== BILAN EXPLICABILITÉ BREAST CANCER ===")
print(f"Nombre de paires testées : {total}")
print(f"Nombre de paires sans explication : {nb_echecs}")
print(f"Proportion de paires non-explicables : {proportion:.2f}%")
print("-" * 50)

Analyse de 100 paires aléatoires...

=== BILAN EXPLICABILITÉ BREAST CANCER ===
Nombre de paires testées : 100
Nombre de paires sans explication : 2
Proportion de paires non-explicables : 2.00%
--------------------------------------------------


On observe une forte explicabilité: 98 % des décisions prises par l'IA sur le cancer du sein peuvent être justifiées par des compensations simples (un avantage qui compense plusieurs inconvénients, ou inversement). Cela montre que le diagnostic n'est pas une boîte noire complexe, mais une balance logique de symptômes.

Les 2 paires inexplicables représentent des cas où la décision de la régression logistique est holistique. C'est-à-dire que le modèle a pris sa décision en additionnant une multitude de micro-facteurs qui, isolés, ne sont pas assez forts pour compenser un gros point négatif, mais qui, tous ensemble, font pencher la balance. 

Comme le modèle d'explication hybride force le regroupement autour d'un "Hub" (soit 1-m, soit m-1), il ne peut pas capturer ces cas de somme globale diffuse.

On teste avec un sampling plus élevé pour vérifier la robustesse de nos résultats:

In [51]:
n_samples = 500
nb_echecs, total, proportion = evaluer_explicabilite_dataset(breastcancer_X_scaled, breastcancer_weights, breastcancer_feature_names, n_samples)

print(f"\n=== BILAN EXPLICABILITÉ BREAST CANCER ===")
print(f"Nombre de paires testées : {total}")
print(f"Nombre de paires sans explication : {nb_echecs}")
print(f"Proportion de paires non-explicables : {proportion:.2f}%")
print("-" * 50)

Analyse de 500 paires aléatoires...

=== BILAN EXPLICABILITÉ BREAST CANCER ===
Nombre de paires testées : 500
Nombre de paires sans explication : 1
Proportion de paires non-explicables : 0.20%
--------------------------------------------------


On cherche maintenant à identifier précisément pourquoi le modèle échoue sur ces paires. 

In [ ]:
def isoler_paires_echec(X, poids, names, n_paires=500):
    """
    Échantillonne aléatoirement des paires de données et identifie celles pour lesquelles 
    le modèle d'explication hybride ne parvient pas à trouver de solution optimale.

    Cette fonction permet de mesurer empiriquement la couverture du langage d'explication 
    (trade-offs 1-m et m-1) sur un dataset complexe comme celui du Breast Cancer.

    Args:
        X (numpy.ndarray ou pd.DataFrame): Les features standardisées.
        poids (list ou numpy.array): Les coefficients du modèle de régression logistique.
        names (list): Liste des noms des features.
        n_paires (int, optionnel): Nombre de paires à tester par échantillonnage aléatoire. 
                                   Par défaut 500.

    Returns:
        list of tuples: Une liste de tuples (idx_superieur, idx_inferieur) contenant les 
                        indices des paires jugées inexplicables par le modèle hybride.
    """
    scores = X @ poids
    indices = list(range(len(X)))
    paires_echec = []
    
    print(f"Analyse de {n_paires} paires...")
    
    found = 0
    while found < n_paires:
        idx_a, idx_b = random.sample(indices, 2)
        if scores[idx_a] >= scores[idx_b]:
            p_sup, p_inf = X[idx_a], X[idx_b]
            indices_paire = (idx_a, idx_b)
        else:
            p_sup, p_inf = X[idx_b], X[idx_a]
            indices_paire = (idx_b, idx_a)
            
        status, _ = explication_hybride(p_sup, p_inf, poids, names)
        
        if status != "OPTIMAL":
            paires_echec.append(indices_paire)
        
        found += 1
    
    return paires_echec

liste_echecs = isoler_paires_echec(breastcancer_X_scaled, breastcancer_weights, breastcancer_feature_names, 500)
print(f"Nombre d'échecs trouvés : {len(liste_echecs)}")

Analyse de 500 paires...
Nombre d'échecs trouvés : 4


In [52]:
def analyser_detail_echec(paire, X, poids, names):
    """
    Réalise une analyse de l'échec de l'explication pour une paire donnée.
    
    Cette fonction décompose les contributions de chaque caractéristique médicale
    pour identifier pourquoi les contraintes de partitionnement (1-m ou m-1) 
    n'ont pas pu être satisfaites par le solveur Gurobi, malgré un score global positif.

    Args:
        paire (tuple): Un tuple (idx_superieur, idx_inferieur) issu de isoler_paires_echec.
        X (numpy.ndarray): Matrice des caractéristiques standardisées.
        poids (list): Vecteur des coefficients du modèle de régression logistique.
        names (list): Noms des variables médicales (sans la colonne cible).

    Returns:
        None: Affiche les tableaux de contributions et les statistiques de synthèse.
    """
    idx_sup, idx_inf = paire
    
    notes_sup = X[idx_sup]
    notes_inf = X[idx_inf]
    contribs, P, C = preparer_donnees(notes_sup, notes_inf, poids)

    df_contribs = pd.DataFrame({
        'Feature': names,
        'Contribution': contribs
    }).sort_values(by='Contribution', ascending=False)

    print(f"\n=== Détail de l'échec (Patient {idx_sup} vs {idx_inf}) ===")
    print("Top Avantages (Pros) :")
    print(df_contribs.head(5).to_string(index=False))
    print("\nPires Inconvénients (Cons) :")
    print(df_contribs.tail(5).to_string(index=False))
    
    pros_sum = sum(contribs[i] for i in P)
    cons_min = min(contribs) if C else 0
    
    print(f"\nSomme totale des avantages : {pros_sum:.4f}")
    print(f"Pire inconvénient individuel : {cons_min:.4f}")
    print("-" * 50)


if liste_echecs:
    for i in range(len(liste_echecs)):
        analyser_detail_echec(liste_echecs[i], breastcancer_X_scaled, breastcancer_weights, breastcancer_feature_names)
else:
    print("Aucun échec trouvé dans cet échantillon.")


=== Détail de l'échec (Patient 441 vs 252) ===
Top Avantages (Pros) :
                 Feature  Contribution
          BlandChromatin      1.169578
          NormalNucleoli      0.987644
SingleEpithelialCellSize      0.224811
    UniformityOfCellSize      0.077884
          ClumpThickness      0.000000

Pires Inconvénients (Cons) :
              Feature  Contribution
       ClumpThickness      0.000000
           BareNuclei      0.000000
     MarginalAdhesion     -0.560698
UniformityOfCellShape     -0.807866
              Mitoses     -0.835164

Somme totale des avantages : 2.4599
Pire inconvénient individuel : -0.8352
--------------------------------------------------

=== Détail de l'échec (Patient 220 vs 120) ===
Top Avantages (Pros) :
         Feature  Contribution
  NormalNucleoli      1.185173
  BlandChromatin      1.169578
MarginalAdhesion      0.560698
  ClumpThickness      0.000000
      BareNuclei      0.000000

Pires Inconvénients (Cons) :
                 Feature  Contribut

L'analyse des paires inexplicables révèle que l'échec ne provient pas d'une contradiction du modèle (la somme des contributions reste positive), mais d'une limite du langage d'explication.

Le modèle hybride impose que chaque argument appartienne à une structure simple (soit un avantage pour plusieurs défauts, soit l'inverse). Dans les cas d'échec rencontrés, les faiblesses médicales du patient sont trop éparpillées: il y a plusieurs inconvénients de taille moyenne qui nécessiteraient une structure de type n-m (plusieurs avantages pour plusieurs inconvénients), ce que le modèle hybride interdit pour rester intelligible.

Cela démontre qu'l existe une frange de diagnostics médicaux où la décision repose sur une interaction complexe et globale de tous les symptômes simultanément, rendant toute simplification par groupes de compensation incomplète.

### 2. Dataset RATP

L'idée est d'expliquer, en utilisant les algorithmes développés, quelles stations de métro est plus prioritaire pour être rénové. 

On commence par faire un scoring en prenant la somme pondérée des features des stations pour avoir une idée de leur classement en terme de priorité de rénovation.

In [ ]:

ratp_data = pd.read_csv('data/ratp_cleaned_data.csv', index_col=0, sep=';')
ratp_weights = [0.021, 0.188, 0.038, 0.322, 16.124, 67.183, 16.124]
ratp_feature_cols = ratp_data.columns.tolist()

stations_dict = {}

for index, row in ratp_data.iterrows():
    station_name = index
    values = row[ratp_feature_cols].values.tolist()
    stations_dict[station_name] = values


# 1. Create a dictionary of just the scores
scores = {
    key: sum(val * w for val, w in zip(values, ratp_weights)) 
    for key, values in stations_dict.items()
}

print("--- Calculated Scores ---")
for key, score in scores.items():
    print(f"Key '{key}': {score:.2f}")


# 3. Sort the original data based on these scores
# We look up the score in the 'scores' dict we just made
sorted_data = dict(sorted(
    stations_dict.items(), 
    key=lambda item: scores[item[0]], 
    reverse=True
))

print("\n--- Final Sorted Data ---")
print(sorted_data)

--- Calculated Scores ---
Key 'Odéon (Ligne 4)': 9484.28
Key 'Place d'Italie (Lign 6)': 9457.61
Key 'Jussieu (Ligne 7)': 9436.20
Key 'Nation (Ligne 9)': 9389.68
Key 'La Motte Picquet-Grenelle (Ligne 10)': 9349.76
Key 'Porte d'Orléans (Ligne 4)': 9328.05
Key 'Daumenil (Ligne 6)': 9144.58
Key 'Vaugirard (Ligne 12)': 9107.66
Key 'Oberkampf (Ligne 9)': 9229.01
Key 'Reuilly-Diderot (Ligne 1)': 9240.69

--- Final Sorted Data ---
{'Odéon (Ligne 4)': [85000.0, 8100.0, 35500.0, 3450.0, 75.0, 16.2, 88.0], "Place d'Italie (Lign 6)": [81000.0, 8100.0, 37500.0, 3150.0, 67.0, 17.6, 95.0], 'Jussieu (Ligne 7)': [74000.0, 8900.0, 37000.0, 4050.0, 68.0, 16.8, 79.0], 'Nation (Ligne 9)': [74000.0, 7100.0, 42000.0, 4550.0, 77.0, 15.2, 73.0], 'La Motte Picquet-Grenelle (Ligne 10)': [72000.0, 7500.0, 33000.0, 4250.0, 88.0, 13.2, 93.0], "Porte d'Orléans (Ligne 4)": [71000.0, 7300.0, 31500.0, 4600.0, 76.0, 15.8, 93.0], 'Reuilly-Diderot (Ligne 1)': [72000.0, 8700.0, 36000.0, 4000.0, 66.0, 16.6, 78.0], 'Oberkamp

On voit alors que c'est la station Odéon qui est la plus susceptible à avoir besoin de rénovation. On cacule 1 à 1, dans l'ordre de la liste, l'explication fournie par notre algorithme (1-m) et (m-1) mélangé.

In [51]:
for i in range(len(ratp_data)-1):
    station_i = ratp_data.index[i]
    station_i1 = ratp_data.index[i+1]
    notes_i = stations_dict[station_i]
    notes_j = stations_dict[station_i1]
    status, result = explication_hybride(notes_i, notes_j, ratp_weights, ratp_feature_cols)
    afficher_resultats(f"DONNÉES RATP : La station {station_i} vs {station_i1}", status, result)


=== DONNÉES RATP : La station Odéon (Ligne 4) vs Place d'Italie (Lign 6) ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage peak-entering-passengers/h (+84.0) compense ['off-peak-entering-passengers/h']
  • Groupe (m-1) : Les avantages ['off-peak-passing-passengers/h'] (+96.60000000000001) compensent Station degradation level ([0,20]  scale) (-94.05620000000015)
  • Groupe (m-1) : Les avantages ['strategic priority [0,10]'] (+128.992) compensent connectivity index [0,100] (-112.868)
--------------------------------------------------

=== DONNÉES RATP : La station Place d'Italie (Lign 6) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Jussieu (Ligne 7) vs Nation (Ligne 9) ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage peak-passing-passengers/h (+338.4) compense ['off-peak-passing-passengers/h', 'strategic

On obtient alors les explicaitons suivantes :
- Odéon -> Place d'Italie 
- Jussieu -> Nation 
- La Motte Picquet-Grenelle -> Porte D'Orléans
- Porte d'Orléans -> Daumesnil
- Daumenil -> Vaugirard

On essaie donc de "boucher les trous évidents" si on en trouve. Par exemple, on essaie d'expliquer que Place D'Italie prime sur Nation.


In [54]:
# Place d'Italie vs Nation
station_place_italie = ratp_data.index[1]
notes_place_italie = stations_dict[station_place_italie]
station_nation = ratp_data.index[3]
notes_nation = stations_dict[station_nation]
status, result = explication_hybride(notes_place_italie, notes_nation, ratp_weights, ratp_feature_cols)
afficher_resultats(f"DONNÉES RATP : La station {station_place_italie} vs {station_nation}", status, result)


# Nation vs Porte d'Orléans
station_nation = ratp_data.index[3]
notes_nation = stations_dict[station_nation]
station_porte_orleans = ratp_data.index[5]
notes_porte_orleans = stations_dict[station_porte_orleans]
status, result = explication_hybride(notes_nation, notes_porte_orleans, ratp_weights, ratp_feature_cols)
afficher_resultats(f"DONNÉES RATP : La station {station_nation} vs {station_porte_orleans}", status, result)

# Odéon vs La motte Picquet
station_odeon = ratp_data.index[0]
notes_odeon = stations_dict[station_odeon]
station_la_motte_picquet = ratp_data.index[4]
notes_la_motte_picquet = stations_dict[station_la_motte_picquet]
status, result = explication_hybride(notes_odeon, notes_la_motte_picquet, ratp_weights, ratp_feature_cols)
afficher_resultats(f"DONNÉES RATP : La station {station_odeon} vs {station_la_motte_picquet}", status, result)


=== DONNÉES RATP : La station Place d'Italie (Lign 6) vs Nation (Ligne 9) ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage connectivity index [0,100] (+354.72799999999995) compense ['off-peak-entering-passengers/h', 'strategic priority [0,10]']
  • Groupe (m-1) : Les avantages ['peak-entering-passengers/h', 'peak-passing-passengers/h', 'Station degradation level ([0,20]  scale)'] (+496.23920000000015) compensent off-peak-passing-passengers/h (-450.8)
--------------------------------------------------

=== DONNÉES RATP : La station Nation (Ligne 9) vs Porte d'Orléans (Ligne 4) ===
SUCCÈS : Une explication a été trouvée :
  • Groupe (1-m) : L'avantage off-peak-entering-passengers/h (+399.0) compense ['Station degradation level ([0,20]  scale)', 'connectivity index [0,100]']
  • Groupe (m-1) : Les avantages ['peak-entering-passengers/h'] (+63.00000000000001) compensent peak-passing-passengers/h (-37.6)
  • Groupe (m-1) : Les avantages ['strategic priority [0,10]

On a alors :
- Odéon prime sur Place d'Italie
- Odéon ne prime pas sur Jussieu pour l'instant
- Odéon prime sur Nation par transitivité car Place d'Italie prime sur Nation
- Odéon prime sur La Motte Picquet
- Odéon prime sur Porte d'Orléans par transitivié car Nation prime sur Porte d'Orléans
- Odéon prime sur Daumesnil par transitivité car Porte d'Orléans prime sur Daumesnil
- Odéon prime sur Vaugirard par transitivité car Daumesnil prime sur Vaugirard
- Odéon ne prime pas sur Oberkampf pour l'instant
- Odéon ne prime pas sur Reuilly Diderot pour l'instant

Afin de procéder avec les trois stations restantes, on essaie de voir quelles stations priment sur eux.

In [55]:
# Jussieu
station_jussieu = ratp_data.index[2]
notes_jussieu = stations_dict[station_jussieu]

for i in range(len(ratp_data)):
    if i == 2:
        continue
    station_i = ratp_data.index[i]
    notes_i = stations_dict[station_i]
    status, result = explication_hybride(notes_i, notes_jussieu, ratp_weights, ratp_feature_cols)
    afficher_resultats(f"DONNÉES RATP : La station {station_i} vs {station_jussieu}", status, result)


# Oberkampf
station_oberkampf = ratp_data.index[8]
notes_oberkampf = stations_dict[station_oberkampf]

for i in range(len(ratp_data)):
    if i == 8:
        continue
    station_i = ratp_data.index[i]
    notes_i = stations_dict[station_i]
    status, result = explication_hybride(notes_i, notes_oberkampf, ratp_weights, ratp_feature_cols)
    afficher_resultats(f"DONNÉES RATP : La station {station_i} vs {station_oberkampf}", status, result)


# Reuilly Diderot
station_reuilly_diderot = ratp_data.index[9]
notes_reuilly_diderot = stations_dict[station_reuilly_diderot]

for i in range(len(ratp_data)):
    if i == 9:
        continue
    station_i = ratp_data.index[i]
    notes_i = stations_dict[station_i]
    status, result = explication_hybride(notes_i, notes_reuilly_diderot, ratp_weights, ratp_feature_cols)
    afficher_resultats(f"DONNÉES RATP : La station {station_i} vs {station_reuilly_diderot}", status, result)


=== DONNÉES RATP : La station Odéon (Ligne 4) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Place d'Italie (Lign 6) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Nation (Ligne 9) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station La Motte Picquet-Grenelle (Ligne 10) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Porte d'Orléans (Ligne 4) vs Jussieu (Ligne 7) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
-------------------------------

On trouve alors les observations suivantes.
- Odéon prime sur Oberkampf par transitivité car Place d'Italie prime sur Oberlampf.
- Odéon prime sur Reuilly Diderot.

Il n'y a aucune station qui prime sur Jussieu.

In [56]:
# Odéon
station_odeon = ratp_data.index[0]
notes_odeon = stations_dict[station_odeon]

for i in range(len(ratp_data)):
    if i == 0:
        continue
    station_i = ratp_data.index[i]
    notes_i = stations_dict[station_i]
    status, result = explication_hybride(notes_i, notes_odeon, ratp_weights, ratp_feature_cols)
    afficher_resultats(f"DONNÉES RATP : La station {station_i} vs {station_odeon}", status, result)


=== DONNÉES RATP : La station Place d'Italie (Lign 6) vs Odéon (Ligne 4) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Jussieu (Ligne 7) vs Odéon (Ligne 4) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Nation (Ligne 9) vs Odéon (Ligne 4) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station La Motte Picquet-Grenelle (Ligne 10) vs Odéon (Ligne 4) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------

=== DONNÉES RATP : La station Porte d'Orléans (Ligne 4) vs Odéon (Ligne 4) ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
---------------------------------------

Il n'y a aucune station qui prime sur Odéon non plus. On en déduit que, bien que Odéon prime sur toutes les autres station, et prime sur Jussieu en terme de somme pondérée des features, on ne peut pas l'expliquer avec nos algorithmes présent.

### 3. Données avec 27 features


L’objectif est d’analyser des comparaisons locales entre solutions à l’aide d’un langage d’explication fondé sur des trade-offs, **sans recourir à un classement global par score**.

Chaque solution est décrite par 27 critères pondérés. Pour une comparaison donnée $x \succ y$, on calcule pour chaque critère $k$ la contribution :

$$
c_k = w_k(x_k - y_k)
$$

Ces contributions permettent de distinguer :
- les arguments favorables : $c_k > 0$,
- les arguments défavorables : $c_k < 0$,
- les arguments neutres : $c_k = 0$.

Les ensembles ainsi obtenus constituent l’entrée du modèle d’explication.

1. Chargement des données (poids + deux premières solutions pour démarrer)

In [17]:
def load_first_two_solutions_27crit(path: str):
    raw = pd.read_excel(path, header=None)

    col0 = raw[0].astype(str).str.strip().str.lower()
    wrow_idx = raw.index[col0 == "weight"][0]

    crit_row_idx = wrow_idx - 1
    criteria = raw.loc[crit_row_idx, 1:].tolist()
    criteria = [str(c).strip() for c in criteria]
    criteria = [c for c in criteria if c and c.lower() != "nan"]

    wvals = raw.loc[wrow_idx, 1: 1 + len(criteria)].tolist()
    weights = [float(w) for w in wvals]

    data = raw.loc[wrow_idx + 1:, :].copy()
    data = data.dropna(how="all")
    data = data.rename(columns={0: "solution"})
    data["solution"] = data["solution"].astype(str).str.strip()

    s1 = data[data["solution"].str.lower() == "solution 1"].iloc[0]
    s2 = data[data["solution"].str.lower() == "solution 2"].iloc[0]

    notes_s1 = [float(s1[i]) for i in range(1, 1 + len(criteria))]
    notes_s2 = [float(s2[i]) for i in range(1, 1 + len(criteria))]

    return criteria, weights, notes_s1, notes_s2

subjects, weights, sol1, sol2 = load_first_two_solutions_27crit("data27crit.xlsx")

2. Explication mixte

Nous testons directement l’existence d’une explication mixte, combinant des trade-offs de type (1–m) et (m–1), pour une comparaison donnée. Aucune hypothèse préalable n’est faite sur la validité de la comparaison.

In [23]:
subjects, weights, sol1, sol2 = load_first_two_solutions_27crit("data/data27crit.xlsx")

In [24]:
status, explanation = explication_hybride(sol1, sol2, weights, subjects)
print("solution 1 > solution 2")
print(status)
for e in explanation:
    print(e)

solution 1 > solution 2
OPTIMAL
Groupe (1-m) : L'avantage a (+876.0) compense ['e']
Groupe (1-m) : L'avantage h (+276.0) compense ['i', 'x']
Groupe (1-m) : L'avantage j (+862.0) compense ['r']
Groupe (1-m) : L'avantage m (+524.0) compense ['A']
Groupe (1-m) : L'avantage n (+968.0) compense ['l', 'p']
Groupe (1-m) : L'avantage o (+380.0) compense ['g']
Groupe (m-1) : Les avantages ['u'] (+314.0) compensent f (-224.00000000000006)
Groupe (m-1) : Les avantages ['c'] (+778.0) compensent k (-496.0)
Groupe (m-1) : Les avantages ['d'] (+926.0) compensent q (-513.9999999999999)
Groupe (m-1) : Les avantages ['s'] (+844.0) compensent t (-826.0)
Groupe (m-1) : Les avantages ['b'] (+876.0) compensent v (-348.0)
Groupe (m-1) : Les avantages ['z'] (+906.0) compensent w (-582.0)


In [25]:
status, explanation = explication_hybride(sol2, sol1, weights, subjects)
print("solution 2 > solution 1")
print(status)

if explanation is not None:
    for e in explanation:
        print(e)
else:
    print("Aucune explication mixte possible.")

solution 2 > solution 1
INFEASIBLE
Aucune explication mixte possible.


Interprétation des résultats

Pour la comparaison solution 1 > solution 2, le solveur retourne le statut OPTIMAL, ce qui indique l’existence d’une explication mixte valide combinant des trade-offs de type (1–m) et (m–1).
La supériorité de la solution 1 peut ainsi être justifiée par des compensations locales entre critères, sans utiliser de score global.

En revanche, pour la comparaison inverse solution 2 > solution 1, le solveur retourne le statut INFEASIBLE.
Cela constitue un certificat de non-existence d’explication dans le langage considéré : aucune combinaison de trade-offs autorisés ne permet de justifier cette supériorité.
